In [ ]:
import whisper, os
import pyttsx3 as tts
import google.generativeai as genai
from dotenv import load_dotenv
import movis as mv
import json

load_dotenv()
GEMINI_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_KEY)

In [ ]:
def path_validation(path: str):
    '''
    This function will validate if the target path is valid
    if not it will create a new folder

    - path: str -> folder you want to validate
    '''
    if not os.path.exists(path):
        os.makedirs(path)
        return

def speak(text: str, filename: str):
    '''
    This function will speak the given text and save the file in MP3 format

    - text: str -> text you want to speak
    - filename: str -> name of the file you want to save
    '''
    engine = tts.init()
    engine.setProperty("rate", 150)

    folder = 'audio'
    path_validation(folder)

    engine.save_to_file(text, f'{folder}/{filename}.mp3')
    engine.runAndWait()

def generate_script(story_prompt: str, story_count: int):
    '''
    This function will generate a script based on the given prompt

    - story_prompt: str -> prompt you want to generate
    - story_count: int -> number of stories you want to generate
    '''
    model = genai.GenerativeModel('gemini-pro')
    default_prompt = """from now on return the output as regular text and the story should look
                    like it being narrated by someone. Write a Short and simple story,
                    You have to write maximum 170 words I want a plain text no symbols just the script, 
                    also the script content must be in 50 seconds duration, dont include any symbols or special characters,"""
    title_prompt = "For this story, I want you to write a 3 to 5 words title"

    story_folder = 'story'
    audio_folder = 'audio'
    
    path_validation(story_folder)

    for i in range(story_count):
        story_content = model.generate_content(f'{default_prompt} {story_prompt}')
        title_content = model.generate_content(f'{title_prompt} {story_content}')

        with open(f'{story_folder}/story_{i}.json', 'w') as file:
            data = json.dumps({"title": title_content.text, "story": story_content.text})
            file.write(data)

        with open(f'{story_folder}/story_{i}.json', 'r') as file:
            text = json.load(file)
            text = text['story']
            text = [word.replace('\n','') for word in text]
            text = ''.join(text)
            print(text)
            speak(text, f'audio_{i}')

In [ ]:
generate_script('Horror Crime Thriller', 2)